# Obtención de datos

## Instalación e importación de librerías necesarias

In [19]:
!pip install PyPDF4 #librería para el manejo de pdf
!pip install tabula-py #permite a los usuarios leer el contenido de una tabla incrustada en un documento PDF
!pip install -U requests #para realizar solicitudes HTTP y la opción -U es para actualizar a la última versión
!pip install -U beautifulsoup4 #permite extraer información de contenido en formato HTML o XM

In [20]:
from bs4 import BeautifulSoup
import codecs
import csv
from datetime import date
from datetime import datetime
import io
import json
import pandas as pd
import PyPDF4
import re
import requests
import tabula
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Lectura de datos en formato CSV

Un fichero **CSV** (acrónimo de **C**omma **S**eparated **V**alues) es un fichero de **texto** que almacena valores en formato tabular separados por **comas**. Dado que el separador **coma** puede crear confusión a la hora de separar valores numéricos que utilizan la coma como delimitador de la parte entera y la decimal, lo habitual es que se utilice como separador el **punto y coma**. También es posible encontrar como delimitadores el **espacio** y el **tabulador** (en este último caso al fichero se le conoce como **TSV**, acrónimo de **T**ab **S**eparated **V**alues).

Aunque podemos utilizar la librería de Python [`csv`](https://docs.python.org/3/library/csv.html), lo cierto es que la librería [`pandas`](https://pandas.pydata.org/) nos ofrece una función [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) que nos permite construir un `DataFrame` de una forma mucho más cómoda, utilizando como fuente un fichero csv local, o bien uno disponible en la web.

Por ejemplo, vamos a cargar un dataset en formato CSV de calidad de vinos del norte de Portugal. Este [dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) ha sido obtenido del repositorio de aprendizaje automático UC Irvine.

Antes de cargar el dataset mediante `pandas`, es conveniente que visualicemos su contenido para indicar a la librería cómo procesarlo:

![csv](https://drive.google.com/uc?export=view&id=1frKb8yGFYv0kckEirjsrbVJtzge_333T)

Existen varios aspectos a destacar del fichero csv visualizado:

* La primera fila contiene una lista con los nombres de cada campo.
* Los valores se encuentran separados por el delimitador punto y coma.
* Las cifras decimales utilizan el punto (y no la coma) como delimitador de cifras decimales.
* Las cadenas se encuentran encapsuladas entre comillas.
* La codificación del archivo es UTF-8. Algunos programas como Notepad++ suelen detectar automáticamente la codificación, si esto no fuera posible, tendríamos que probar con las codificaciones más conocidas: utf-8, ascii, latin-1, etc.

Todos estos aspectos que hemos mencionado se traducen en parámetros que tendremos que indicar a la función `read_csv` para que realice una correcta interpretación del contenido del fichero:

In [21]:
wine_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", # URL de archivo csv
                      header=0, # La fila número 0 contiene el nombre de las columnas
                      sep=";", # Separador o delimitador de campos
                      decimal=".", # El punto separa la parte entera de la decimal de un número
                      quotechar="\"", # Las cadenas se encuentran encapsuladas entre comillas
                      encoding='utf-8',  # La codificación del fichero es UTF-8
)
wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


La función `read_csv` define muchos más [parámetros](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), aunque su uso no es tan común. Además, es interesante conocer la lista de [codificaciones reconocidas por Python](https://docs.python.org/3/library/codecs.html#standard-encodings). 

## Lectura de datos en formato JSON


El formato **JSON** (acrónimo de **J**ava**S**cript **O**bject **N**otation) es un formato ampliamente adoptado por las bases de datos documentales. Se trata de una alternativa más concisa al formato **XML**.

Este formato nos da soporte para representar tipos de datos *numéricos*, *booleanos*, *cadenas* y la ausencia de valor mediante el término `null`. Permite la definición de estructuras complejas como *arrays* (listas ordenadas de elementos) y *objetos* (diccionarios de pares **clave**, **valor**), los cuales se pueden anidar indefinidamente y no imponen restricciones en cuanto al tipo de dato almacenado.

Veamos un ejemplo:

```json
{
  "numero": 1.23456,
  "cadena": "Hola Mundo!",
  "array": [1, 
            "texto", 
            {"campo": "valor", "otros": [1,2,3]}, 
            null],
  "objeto": {
    "anidado": {
      "anidado": {
        "anidado": {}
      }
    }
  }
}
```

Además de ser un formato tan extendido como el **CSV**. Suele ser el formato de respuesta utilizado por las aplicaciones **REST** (acrónimo de **RE**presentational **S**tate **T**ransfer). Estas aplicaciones implementan una interfaz basada en el protocolo **HTTP** para la obtención y manipulación remota de datos de manera sencilla e independiente de la tecnología empleada.

A continuación vamos a leer los datos oficiales de población de los diferentes municipios de la Región de Murcia a través del [Instituto Nacional de Estadística](https://www.ine.es/index.htm) para el año 2019. Aunque existen otros métodos para leer documentos JSON mediante Python (por ejemplo, la librería [json](https://docs.python.org/3/library/json.html)), nosotros utilizaremos el método [`read_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html) ofrecido por `pandas`.

Este método define una serie de [parámetros](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html) con los que controlar la forma en que el documento JSON es interpretado, aunque generalmente no necesitaremos indicarlos y sus valores por defecto serán adecuados:

In [22]:
#pob_df = pd.read_json("https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/2883?tip=AM&tv=18:451")
pob_df = pd.read_json("https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/2883?tip=AM&tv=18:451&date=20190101")
pob_df

,COD,Nombre,T3_Unidad,T3_Escala,MetaData,Data
0,DPOP13765,Murcia. Total. Total habitantes. Personas.,Personas,,"[{'Id': 31, 'Variable': {'Id': 115, 'Nombre': ...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
1,DPOP13768,Abanilla. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2640, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
2,DPOP13771,Abarán. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2641, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
3,DPOP13774,Águilas. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2642, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
4,DPOP13777,Albudeite. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2643, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
5,DPOP13780,Alcantarilla. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2526, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
6,DPOP13900,"Alcázares, Los. Total. Total habitantes. Perso...",Personas,,"[{'Id': 2248, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
7,DPOP13783,Aledo. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2527, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
8,DPOP13786,Alguazas. Total. Total habitantes. Personas.,Personas,,"[{'Id': 2528, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."
9,DPOP13789,Alhama de Murcia. Total. Total habitantes. Per...,Personas,,"[{'Id': 2529, 'Variable': {'Id': 19, 'Nombre':...","[{'Fecha': '2019-01-01T00:00:00.000+01:00', 'T..."


Como podréis observar, el conjunto de datos obtenido necesita mucho preprocesamiento para ser comprensible por un ser humano.

## Lectura de datos en formato PDF

Existen varias librerías para la extracción y manipulación de los datos de archivos PDF, como por ejemplo: [PDFMiner](https://pypi.org/project/pdfminer/), [slate](https://pypi.org/project/slate/), [PyPDF2](https://pypi.org/project/PyPDF2/) y [PyPDF4](https://pypi.org/project/PyPDF4/). Nosotros utilizaremos PyPDF4, aunque las características proporcionadas por estas librerías son similares.

A diferencia de los ejemplos vistos anteriormente, esta librería no permite la lectura directa de PDFs de la web, por lo que tendremos que utilizar otras librerías para recuperar su contenido. En nuestro caso haremos uso de la librería [`urllib3`](https://urllib3.readthedocs.io/en/latest/), un popular un cliente HTTP que nos resolverá las peticiones a los recursos que deseemos descargar.

El siguiente código emite una petición GET de lectura a una URL facilitada por el [Ministerio de Sanidad, Consumo y Bienestar Social](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/situacionActual.htm) mediante la que descargaremos un PDF que informará sobre la situación del COVID-19 a fecha de 24/12/2020:

In [23]:
PDF_URL = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_278_COVID-19.pdf'

In [24]:
http = urllib3.PoolManager()
req = http.request('GET', PDF_URL)

Si los datos se han recuperado correctamente, observaremos un valor de `200` en el campo `status` del resultado de la petición que acabamos de realizar:

In [25]:
req.status

200

La librería `PyPDF4` necesita un **stream** de datos para obtener un PDF. Si trabajáramos con un fichero local, haríamos uso de la función [open](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) de Python para obtener este stream, sin embargo, ya que tenemos el contenido cargado en **memoria**, haremos uso de la librería `io` para su obtención:

In [26]:
pdf = PyPDF4.pdf.PdfFileReader(io.BytesIO(req.data))

Una vez cargado el fichero pdf, tendremos acceso a una serie de metadatos sobre el archivo, como por ejemplo su autor o la herramienta con la que fue creado:

In [27]:
info = pdf.documentInfo
print(f"Autor: {info.author}")
print(f"Creado con: {info.creator}")

Autor: Jesús Pérez Formigó
Creado con: Microsoft® Word 2010


También tenemos acceso a las diferentes páginas que componen el PDF:

In [28]:
page = pdf.getPage(0)

Esta librería nos proporciona el método `extractText` mediante el que obtener el texto de una determinada página:

In [29]:
text = page.extractText()

Mediante [expresiones regulares](https://docs.python.org/3/library/re.html) podremos, por ejemplo, procesar este texto para obtener datos de interés del documento procesado:

In [30]:
# Esta expresión extrae el total de casos confirmados de la página
pattern = '\s+([\d.]+)\s+casos'
match = re.search(pattern, text)
casos_confirmados = match.group(1)
casos_confirmados

'1.854.951'

A veces, los PDFs datos en formato tabular, cuya extración puede resultar tediosa si hacemos uso de las librerías anteriores. Para estos casos, es conveniente utilizar la librería `tabula-py`:

In [31]:
tabs = tabula.read_pdf(PDF_URL, 
    # Este argumento indica las páginas a procesar del PDF, le indicamos que todas
    pages='all' 
)

El método `read_pdf` tratará de procesar el documento PDF y buscará las tablas en él definidas. Finalmente nos recuperará una `lista` de `DataFrames` de `pandas` con el contenido extraído. Observemos la primera tabla obtenida del PDF, con la información de los casos diagnosticados totales de COVID-19 por comunidad autónoma:

In [32]:
tabs[0]

,Unnamed: 0,Unnamed: 1,Casos,Casos diagnosticados en los,Casos diagnosticados en los.1,Casos diagnosticados con fecha de,Casos diagnosticados con fecha de.1
0,NaN,Casos,NaN,NaN,NaN,NaN,NaN
1,CCAA,NaN,diagnosticados,últimos 14 días,últimos 7 días,inicio de síntomas en los últimos 14d.,inicio de síntomas en los últimos 7d.
2,NaN,totales,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,el día previo,No IA*,No IA*,No IA*,No IA*
4,Andalucía,257.755,296,"12.615 149,92","5.351 63,59","4.220 50,15","941 11,18"
5,Aragón,78.124,140,"2.830 214,51","1.200 90,96","1.435 108,77","527 39,95"
6,Asturias,26.565,95,"1.472 143,92","588 57,49","83 8,11","31 3,03"
7,Baleares,32.320,117,"5.453 474,40","2.746 238,89","3.579 311,36","1.234 107,35"
8,Canarias**,25.532,1,"2.809 130,45","1.247 57,91","1.204 55,91","302 14,02"
9,Cantabria,17.398,74,"973 167,45","425 73,14","249 42,85","103 17,73"


Aunque la tabla mostrada no tiene un formato perfecto, la librería nos ha simplificado enormemente la extracción de datos del fichero PDF.

## Lectura de datos desde hojas de cálculo

Una hoja de cálculo es un documento que almacena tanto datos numéricos como alfanuméricos en formato tabular. Se compone de celdas dispuestas como una matriz de filas y columnas, y tiene capacidad para incluir cálculos complejos mediante fórmulas y gráficas.

La librería `pandas` también nos ofrece un método, `read_excel` mediante el que obtener un `DataFrame` a partir de una hoja de cálculo en formato `xls`, `xlsx`, `xlsm`, `xlsb`, `odf`, `ods` y `odt`. A continuación vamos a procesar una hoja de cálculo en formato Excel (xlsx) utilizada como ejemplo de la [web de Microsoft para Power BI](https://docs.microsoft.com/es-es/power-bi/create-reports/sample-financial-download):

In [33]:
pd.read_excel("https://go.microsoft.com/fwlink/?LinkID=521962",
              sheet_name=0,
              header=0,
              thousands=None)

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3,20,32370.0,0.00,32370.00,16185.0,16185.00,2014-01-01,1,January,2014
1,Government,Germany,Carretera,None,1321.0,3,20,26420.0,0.00,26420.00,13210.0,13210.00,2014-01-01,1,January,2014
2,Midmarket,France,Carretera,None,2178.0,3,15,32670.0,0.00,32670.00,21780.0,10890.00,2014-06-01,6,June,2014
3,Midmarket,Germany,Carretera,None,888.0,3,15,13320.0,0.00,13320.00,8880.0,4440.00,2014-06-01,6,June,2014
4,Midmarket,Mexico,Carretera,None,2470.0,3,15,37050.0,0.00,37050.00,24700.0,12350.00,2014-06-01,6,June,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Small Business,France,Amarilla,High,2475.0,260,300,742500.0,111375.00,631125.00,618750.0,12375.00,2014-03-01,3,March,2014
696,Small Business,Mexico,Amarilla,High,546.0,260,300,163800.0,24570.00,139230.00,136500.0,2730.00,2014-10-01,10,October,2014
697,Government,Mexico,Montana,High,1368.0,5,7,9576.0,1436.40,8139.60,6840.0,1299.60,2014-02-01,2,February,2014
698,Government,Canada,Paseo,High,723.0,10,7,5061.0,759.15,4301.85,3615.0,686.85,2014-04-01,4,April,2014


Aunque son múltiples los parámetros de este método documentados en la [web de `pandas`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html), los más relevantes son los siguientes:

* `sheet_name`: puede tomar como valor una cadena, un entero, una lista de cadenas y enteros o `None`. Si su valor es una cadena, cargará la hoja con el nombre indicado, si es un entero cargará la hoja correspondiente a la posición indicada, si es una lista cargará las hojas especificadas en el array y, finalmente, si su valor es `None`, cargará todas las hojas del documento.
* `header`: fila que contiene los nombres de columna de los datos tabulados.
* `thousands`: separador de decimales para aquellas celdas de tipo texto en la hoja de cálculo original que desean ser cargadas como datos de tipo numérico.

## Uso de APIs REST para la obtención de datos:

En ocasiones no se nos proporciona directamente el fichero con los datos a analizar, sino que en su lugar se nos facilita una URL a través de la cual un servicio escucha peticiones de los usuarios y proporciona los datos solicitados. Estos servicios suelen proporcionarnos una API REST con la que podremos interactuar mediante los distintos tipos de mensajes HTTP (GET, POST, PUT, DELETE, PATCH, etc).

En estos casos, tendremos que utilizar una librería que implemente el protocolo HTTP. En Python tenemos varias opciones ([`urllib`](https://docs.python.org/3/library/urllib.html), [`httplib2`](https://pypi.org/project/httplib2/),[`urllib3`](https://urllib3.readthedocs.io/en/latest/) ,[`requests`](https://requests.readthedocs.io/en/master/), [`grequests`](https://pypi.org/project/grequests/), [`aiohttp`](https://docs.aiohttp.org/en/stable/), etc). Nosotros haremos uso de `urllib3`, aunque la librería `requests` es también una gran opción.



Por ejemplo, vamos a hacer uso de la API rest implementada en https://restcountries.eu/rest/v2/ para obtener información sobre todos los países:

El siguiente fragmento de código accede a una API REST de citas célebres y nos recupera frases de diferentes autores célebres:



In [36]:
req = http.request('GET', 'https://quote-garden.onrender.com/api/v3/quotes')
req.data

b'{"statusCode":200,"message":"Quotes","pagination":{"currentPage":1,"nextPage":2,"totalPages":7268},"totalQuotes":72672,"data":[{"_id":"5eb17aadb69dc744b4e70d4a","quoteText":"Forty is the old age of youth, fifty is the youth of old age.","quoteAuthor":"Hosea Ballou","quoteGenre":"age","__v":0},{"_id":"5eb17aadb69dc744b4e70d5c","quoteText":"I\'m not interested in age. People who tell me their age are silly. You\'re as old as you feel.","quoteAuthor":"Henri Frederic Amiel","quoteGenre":"age","__v":0},{"_id":"5eb17aadb69dc744b4e70d9c","quoteText":"A man loves the meat in his youth that he cannot endure in his age.","quoteAuthor":"William Shakespeare","quoteGenre":"age","__v":0},{"_id":"5eb17aadb69dc744b4e70d3c","quoteText":"Age considers youth ventures.","quoteAuthor":"Rabindranath Tagore","quoteGenre":"age","__v":0},{"_id":"5eb17aadb69dc744b4e70d2c","quoteText":"Nobody grows old merely by living a number of years. We grow old by deserting our ideals. Years may wrinkle the skin, but to g

En https://restcountries.eu/#api-endpoints se nos proporciona información para conseguir consultas más elaboradas sobre esta API de países, por ejemplo, para consultar por capital, por idioma o por continente.

En este caso, la respuesta no está en un formato que podamos traducir directamente a un `DataFrame` de `pandas`, ya que incluye metadatos que no son de nuestro interés. Haremos uso de la librería `json` para procesar la respuesta y construir el DataFrame a partir de los datos (no metadatos) de la respuesta:

In [37]:
# Convertimos datos de respuesta en un stream de bytes
stream = io.BytesIO(req.data) 
# Traducimos el stream de bytes con contenido JSON a estructuras Python
content = json.load(stream)
# Construimos un DataFrame con el campo data de la respuesta obtenida
pd.DataFrame(content['data'])

,_id,quoteText,quoteAuthor,quoteGenre,__v
0,5eb17aadb69dc744b4e70d4a,"Forty is the old age of youth, fifty is the yo...",Hosea Ballou,age,0
1,5eb17aadb69dc744b4e70d5c,I'm not interested in age. People who tell me ...,Henri Frederic Amiel,age,0
2,5eb17aadb69dc744b4e70d9c,A man loves the meat in his youth that he cann...,William Shakespeare,age,0
3,5eb17aadb69dc744b4e70d3c,Age considers youth ventures.,Rabindranath Tagore,age,0
4,5eb17aadb69dc744b4e70d2c,Nobody grows old merely by living a number of ...,Samuel Ullman,age,0
5,5eb17aadb69dc744b4e70d33,Every man over forty is a scoundrel.,George Bernard Shaw,age,0
6,5eb17aadb69dc744b4e70d49,Old minds are like old horses you must exercis...,John Adams,age,0
7,5eb17aadb69dc744b4e70d5b,You end up as you deserve. In old age you must...,Judith Viorst,age,0
8,5eb17aadb69dc744b4e70d34,Forty is the old age of youth fifty the youth ...,Victor Hugo,age,0
9,5eb17aadb69dc744b4e70d55,I think your whole life shows in your face and...,Lauren Bacall,age,0


En https://pprathameshmore.github.io/QuoteGarden/ se nos proporciona información para la consulta de citas en base a criterios diferentes a los seleccionados.

Finalmente, existen otras APIs muy conocidas como la de [Reddit](https://www.reddit.com/dev/api/) o [Twitter](https://developer.twitter.com/en/docs/twitter-api). Estas APIs nos exigen identificarnos (generalmente mediante [`OAuth2`](https://oauth.net/2/)), ya que tenemos que seguir unas políticas de uso de sus servicios bastante estrictas. Se recomienda el uso de la librería [`requests-oauthlib`](https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html) para estos escenarios.

Además, existen múltiples librerías en la comunidad Python para facilitarnos el uso de las APIs más conocidas, abstrayéndonos de las peticiones HTTP y, en muchos casos, de las restricciones de uso impuestas. A continuación incluimos algunos ejemplos:

* Reddit: [`PRAW`](https://praw.readthedocs.io/en/latest/).
* Twitter: [`searchtweets`](https://twitterdev.github.io/search-tweets-python/), [`python-twitter`](https://python-twitter.readthedocs.io/en/latest/), [`TwitterAPI`](http://geduldig.github.io/TwitterAPI/), [`Tweepy`](http://docs.tweepy.org/en/latest/), [`Twython`](https://twython.readthedocs.io/en/latest/).

## Lectura de datos de páginas web

A veces no tenemos otras vías de recuperar los datos, o se nos imponen limitaciones de acceso demasiado restrictivas, o simplemente el acceso a los recursos es económicamente costoso. Ante estas situaciones, la extracción de los datos de una o varias **páginas web públicamente accesibles** de manera automática puede ser nuestra única opción. A este hecho se le conoce como **Web Scraping**.

Para hacer **Web Scraping** tendremos que trabajar con el protocolo de nivel de aplicación **HTTP** (acrónimo de **H**yper**T**ext **T**ransfer **P**rotocol), que establece las reglas a seguir para permitir la transmisión de datos por la web. Existen múltiples librerías tanto en Python como en otros lenguajes de programación que nos simplifican el uso de este protocolo. En Python, como ya hemos dicho anteriormente, son bien conocidas las siguientes: [`urllib`](https://docs.python.org/3/library/urllib.html), [`httplib2`](https://pypi.org/project/httplib2/),[`urllib3`](https://urllib3.readthedocs.io/en/latest/), [`requests`](https://requests.readthedocs.io/en/master/), [`grequests`](https://pypi.org/project/grequests/), [`aiohttp`](https://docs.aiohttp.org/en/stable/).

En esta sección vamos a trabajar con la librería `requests`, que, como podréis comprobar, ofrece una forma de trabajar muy parecida a la librería `urllib3` que hemos utilizado previamente.

En el siguiente ejemplo vamos a recuperar el contenido de una página de la tienda [catalogoreina.com](https://catalogoreina.com/921-espejos-para-bano) que nos muestra una lista de los espejos ofertados actualmente:

In [ ]:
URL = "https://catalogoreina.com/921-espejos-para-bano"
r = requests.get(URL)
r.text

Como podréis comprobar, el contenido, aunque ligeramente comprensible por el ojo humano, dista mucho de ser manejable. Para facilitar la manipulación de su contenido, haremos uso de la librería `BeautifulSoup`, que implementa un *parser* tanto de páginas web en formato HTML como de ficheros XML:

In [ ]:
html_soup = BeautifulSoup(r.text)

De entre los múltiples métodos que nos ofrece esta librería, destacamos 3 para la recuperación de elementos de una página web:

* `find`: recupera un único elemento de la página web. Define los siguientes parámetros:
  * `name`: tipo de etiqueta a buscar (`a`, `img`, `h1`, `h2`, `article`, etc).
  * `attr`: diccionario que contendrá una lista de atributos y valores (`id`, `class`, `lang`, `title`, etc) que deben coincidir en el elemento recuperado.
  * `recursive`: booleano que indica si la búsqueda se hará recursivamente en los hijos de cada elemento del documento HTML.

    Por ejemplo, vamos a utilizar este método para recuperar el nombre de la categoría de los productos mostrados en la tienda:

In [ ]:
tag = html_soup.find(name='span', attrs={'class': 'cat-name'})
tag

     
Si quisiéramos recuperar el texto contenido en la etiqueta haremos uso del método `get_text` con el parámetro `strip` con valor a `True`, mediante el que quitaremos los espacios en blanco antes y después del texto:

In [ ]:
tag.get_text(strip=True)

* `find_all`: este método es muy parecido a `find`, pero en lugar de un solo resultado nos devolverá una lista de ellos. Define los mismos parámetros que `find` y, además, el parámetro `limit` para limitar el número de resultados obtenido.

    Para mostrar la utilidad de este método, recuperaremos la lista de los precios de tres artículos visualizados en la página:

In [ ]:
tags = html_soup.find_all(name='span', attrs={'class': 'price product-price'}, limit=3)
[ tag.get_text(strip=True) for tag in tags]

Conforme utilicemos la librería, y sobre todo si tenemos experiencia en el manejo de [selectores CSS](https://developer.mozilla.org/es/docs/Web/CSS/CSS_Selectors), nos daremos cuenta de que estos métodos presentan bastantes limitaciones para las complejas reglas de selección que deseamos implementar. Para estas situaciones disponemos de los métodos `select_one` y `select`, que aceptarán un selector CSS y nos devolverán uno o una lista de elementos HTML respectivamente.

Por ejemplo, vamos a recuperar, haciendo uso de selectores CSS, los números de página mostrados en la página inicial (dado que el selector de página aparece dos veces, podremos comprobar que los resultados aparecen repetidos):

In [ ]:
[elem.get_text(strip=True) for elem in html_soup.select('.pagination li')]

Si queremos automatizar la extracción de artículos de la página tendremos que obtener las URLs todas las páginas mostradas. Para ello, extraeremos el valor del atributo `href` de los enlaces de cada página mediante el método `get`:

In [ ]:
links = html_soup.select('.pagination li a')
set([elem.get('href') for elem in links])

De esta forma podríamos construir todas las URLs necesarias para obtener la información de todos los espejos ofertados por la tienda.

Podemos seguir invocando los métodos `find`, `find_all`, `select` y `select_all` sobre cada uno de los resultados obtenidos en búsquedas previas:

In [ ]:
[ link.find('span') for link in links]